<a href="https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/transfer_learning_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

In this notebook, we load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [1]:
from tensorflow.keras.applications.vgg19 import VGG19 # new!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator # new!

#### Load the pre-trained VGG19 model

In [2]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [4]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 50178     
Total params: 20,074,562
Trainable params: 50,178
Non-trainable params: 20,024,384
_________________________________________________________________


#### Compile the model for training

In [6]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

In [7]:
# You can comment out the two lines of code below if you executed them on
# a previous run of the notebook. The wget command downloads the data and the 
# tar command extracts the data from a compressed file format. 
! wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
! tar -xzf hot-dog-not-hot-dog.tar.gz

--2019-11-02 23:25:21--  https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.1, 2620:100:6019:1::a27d:401
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz [following]
--2019-11-02 23:25:21--  https://www.dropbox.com/s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4a717e1730b99ed1b4a7d6dd52.dl.dropboxusercontent.com/cd/0/inline/AroYhyN-Q2Nd3wYkqo_wow1FwY6LX9WSjCxPd4gMXizjSlLge8y6v3Jybke99exhKw-9pJwAJRwrWEZCv2l_w0inZqWpNWxfaDeV1VQZVx-Y_XAPv1tY1gm7UkrqyvnYk_U/file# [following]
--2019-11-02 23:25:21--  https://uc4a717e1730b99ed1b4a7d6dd52.dl.dropboxusercontent.com/cd/0/inline/AroYhyN-Q2Nd3wYkqo_wow1FwY6LX9WSjCxPd4gMXizjSlLge8y6v3Jybke99exhKw-9pJwA

In [8]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [9]:
# Define the batch size:
batch_size=32

In [10]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [11]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/16
15/15 [==============================] - 259s 17s/step - loss: 1.1257 - accuracy: 0.5665 - val_loss: 0.6954 - val_accuracy: 0.6292
Epoch 2/16
15/15 [==============================] - 259s 17s/step - loss: 0.9458 - accuracy: 0.6373 - val_loss: 0.4744 - val_accuracy: 0.7625
Epoch 3/16
15/15 [==============================] - 259s 17s/step - loss: 0.7684 - accuracy: 0.6738 - val_loss: 1.3607 - val_accuracy: 0.5437
Epoch 4/16
15/15 [==============================] - 258s 17s/step - loss: 0.6215 - accuracy: 0.7253 - val_loss: 0.4775 - val_accuracy: 0.7854
Epoch 5/16
15/15 [==============================] - 258s 17s/step - loss: 0.5768 - accuracy: 0.7618 - val_loss: 0.8703 - val_accuracy: 0.6625
Epoch 6/16
15/15 [==============================] - 265s 18s/step - loss: 0.5856 - accuracy: 0.7542 - val_loss: 0.4466 - val_accuracy: 0.8021
Epoch 7/16
15/15 [==============================] - 253s 17s/step - loss: 0.4122 - accuracy: 0.8031 - val_loss: 0.4646 - val_accuracy: 0.7958
Epoch 